## 필요 라이브러리 호출

In [2]:
import pandas as pd
import numpy as np
from selenium import webdriver
from bs4 import BeautifulSoup
from html_table_parser import parser_functions as parser
import time
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import export_graphviz
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from keras.utils import to_categorical

Using TensorFlow backend.
C:\Anaconda3\envs\nlp\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Anaconda3\envs\nlp\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Anaconda3\envs\nlp\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Anaconda3\envs\nlp\lib\site-packages\tensorflow\python\framework\dtypes.py:529: Future

## 데이터 크롤링

In [6]:
# 셀레니움(selenium)으로 브라우저를 컨트를하기 위해서는 webdriver를 설치해야 한다
# 구글크롬은 chromedriver로 검색하면 exe파일을 다운로드 할 수 있다.
# webdriver 설치 위치를 정의한다.
browser = webdriver.Chrome('C:\Program Files (x86)\Google\Chrome\Application\chromedriver')

def crawling(url):
    # 셀레니움(selenium)으로 브라우저를 컨트를하기 위해서는 webdriver를 설치해야 한다
    # 구글크롬은 chromedriver로 검색하면 exe파일을 다운로드 할 수 있다.
    # webdriver 설치 위치를 정의한다.
    browser = webdriver.Chrome('C:\Program Files (x86)\Google\Chrome\Application\chromedriver')
    # 불러오고자하는 url을 .get으로 호출한다.
    browser.get(url)
    
    # page_source파라미터를 이용하면 HTML정보를 가지고 온다.
    html = browser.page_source
    soup = BeautifulSoup(html, 'html.parser')   # 뷰티풀숩으로 HTML을 파싱하고 필요한 데이터 수집
    table_tags = soup.find_all("table")   # find_all함수를 이용하면, TABLE태그로 지정된 곳만 뽑아서, 배열 형태로 저장
    table = table_tags[0]    # html상에서 table 순서, 첫번째 테이블 가지고 와야 함.
    p=parser.make2d(table)

    df_total=pd.DataFrame(p[1:],columns=p[0])    # 데이터 프레임으로 저장

    num = soup.find_all("button")[5].text   # 5번째 button에 페이지수 정보가 있음
    for j in range(1, int(num)):
        btn = browser.find_element_by_class_name('paginationWidget-next')
        btn.click()     # 버튼 클릭
        time.sleep(5)

        # 2페이지 이후 데이터 병합
        html = browser.page_source
        soup = BeautifulSoup(html, 'html.parser')
        table_tags = soup.find_all("table")
        table = table_tags[0]
        p = parser.make2d(table)

        df = pd.DataFrame(p[1:], columns=p[0])
        df_total = pd.concat([df_total, df], 0)
        
    return df_total
        # 엑셀로 다운로드
        # df_total.to_excel("test.xlsx")

In [3]:
# 마이너리그 데이터 크롤링
browser = webdriver.Chrome('C:\Program Files (x86)\Google\Chrome\Application\chromedriver')

def crawling_miner(url):
    # 셀레니움(selenium)으로 브라우저를 컨트를하기 위해서는 webdriver를 설치해야 한다
    # 구글크롬은 chromedriver로 검색하면 exe파일을 다운로드 할 수 있다.
    # webdriver 설치 위치를 정의한다.
    browser = webdriver.Chrome('C:\Program Files (x86)\Google\Chrome\Application\chromedriver')
    # 불러오고자하는 url을 .get으로 호출한다.
    browser.get(url)
    
    # page_source파라미터를 이용하면 HTML정보를 가지고 온다.
    html = browser.page_source
    soup = BeautifulSoup(html, 'html.parser')   # 뷰티풀숩으로 HTML을 파싱하고 필요한 데이터 수집
    table_tags = soup.find_all("table")   # find_all함수를 이용하면, TABLE태그로 지정된 곳만 뽑아서, 배열 형태로 저장
    table = table_tags[0]    # html상에서 table 순서, 첫번째 테이블 가지고 와야 함.
    p=parser.make2d(table)

    df_total_miner=pd.DataFrame(p[1:],columns=p[0])    # 데이터 프레임으로 저장
    
    return df_total_miner


In [7]:
list_url = ["http://mlb.mlb.com/stats/sortable.jsp?c_id=mlb#elem=%5Bobject+Object%5D&tab_level=child&click_text=Sortable+Player+hitting&game_type='R'&season=2019&season_type=ANY&league_code='MLB'&sectionType=sp&statType=hitting&page=1&ts=1580532864437&playerType=ALL",
"http://mlb.mlb.com/stats/sortable.jsp?c_id=mlb#elem=%5Bobject+Object%5D&tab_level=child&click_text=Sortable+Player+fielding&game_type='R'&season=2019&season_type=ANY&league_code='MLB'&sectionType=sp&statType=fielding&page=1&ts=1580539033522&playerType=ALL"]

df_total_hit = crawling(list_url[0])
df_total_field = crawling(list_url[1])

In [4]:
list_url = "http://www.milb.com/milb/stats/stats.jsp?sid=milb&t=l_bat&lid=117"
df_total_hit_miner = crawling_miner(list_url)

In [7]:
df_total_hit_miner

## 전처리 진행
- 불필요한 컬럼 삭제(G, GS) , RK, Team, 빈 컬럼, 등
- hit & field data mergy : key column Player & Team $ Position
- 1B, 2B, 3B, SS 포지션만 추출
- AB 타석수(20), INN(20)가 일정 값 이상의 선수만 대상으로 한다.


In [8]:
# 컬럼 rename
df_total_hit = df_total_hit.rename(columns={'RK':'RK', 'Player':'Player', 'Team':'Team', 
                                            '':'del', '▲':'del2', 'Pos':'Pos', 'G▼':'G', 'AB▼':'AB', 'R▼':'R',
                                            'H▼':'H', '2B▼':'2B', '3B▼':'3B', 'HR▼':'HR',
                                            'RBI▼':'RBI', 'BB▼':'BB', 'SO▼':'SO', 'SB▼':'SB',
                                            'CS▼':'CS', 'AVG▼':'AVG', 'OBP▼':'OBP',
                                            'SLG▼':'SLG', 'OPS▼':'OPS', 'IBB▼':'IBB',
                                            'HBP▼':'HBP', 'SAC▼':'SAC', 'SF▼':'SF', 
                                            'TB▼':'TB', 'XBH▼':'XBH', 'GDP▼':'GDP',
                                            'GO▼':'GO', 'AO▼':'AO', 'GO_AO▼':'GO_AO',
                                            'NP▼':'NP', 'PA▼':'PA'})
df_total_field = df_total_field.rename(columns={'RK':'RK', 'Player':'Player', 'Team':'Team', '':'del',
                                            '▲':'del2', 'Pos':'Pos', 'G▼':'G', 'GS▼':'GS',
                                            'INN▼':'INN', 'TC▼':'TC', 'PO▼':'PO', 'A▼':'A',
                                            'E▼':'E', 'DP▼':'DP', 'SB▼':'SB', 'CS▼':'CS',
                                            'SBPCT▲':'SBPCT', 'PB▼':'PB', 'C_WP▼':'C_WP',
                                            'FPCT▼':'FPCT', 'RF▼':'RF'})

NameError: name 'df_total_hit' is not defined

In [9]:
df_total_hit_miner = df_total_hit_miner.rename(columns={'NAME':'Player', 'TEAM':'Team',
                                            '':'del', '▲':'del2', 'POS':'Pos', 'G▼':'G',
                                            'AB▼':'AB', 'R▼':'R',
                                            'H▼':'H', '2B▼':'2B', '3B▼':'3B', 'HR▼':'HR',
                                            'RBI▼':'RBI', 'TB▼':'TB', 'BB▼':'BB', 'SO▼':'SO', 'SB▼':'SB',
                                            'CS▼':'CS', 'AVG▼':'AVG', 'OBP▼':'OBP',
                                            'SLG▼':'SLG', 'OPS▼':'OPS'})

In [9]:
df_total_hit.to_excel("df_total_hit.xlsx")
df_total_hit

,RK,Player,Team,del,del2,Pos,G,AB,R,H,...,SAC,SF,TB,XBH,GDP,GO,AO,GO_AO,NP,PA
0,1,"Alaniz, R",CIN,,595798,P,8,1,0,1,...,0,0,1,0,0,0,0,-.--,5,1
1,1,"Brebbia, J",STL,,605154,P,63,1,1,1,...,0,0,1,0,0,0,0,-.--,1,1
2,1,"Drake, O",TB,,543118,P,3,1,0,1,...,0,0,1,0,0,0,0,-.--,4,1
3,1,"Liriano, F",PIT,,434538,P,67,1,0,1,...,0,0,1,0,0,0,0,-.--,15,2
4,1,"Lugo, S",NYM,,607625,P,57,1,0,1,...,0,0,1,0,0,0,0,-.--,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32,984,"Yates, K",SD,,489446,P,59,0,0,0,...,0,0,0,0,0,0,0,-.--,0,0
33,984,"Ynoa, G",BAL,,593679,P,4,0,0,0,...,0,0,0,0,0,0,0,-.--,0,0
34,984,"Ynoa, H",ATL,,660623,P,2,0,0,0,...,0,0,0,0,0,0,0,-.--,0,0
35,984,"Zamora, D",NYM,,623354,P,17,0,0,0,...,0,0,0,0,0,0,0,-.--,0,0


In [10]:
df_total_field.to_excel("df_total_field.xlsx")
df_total_field

,RK,Player,Team,del,del2,Pos,G,GS,INN,TC,...,A,E,DP,SB,CS,SBPCT,PB,C_WP,FPCT,RF
0,1,"Abreu, B",HOU,,650556,P,7,0,8.2,4,...,2,0,0,-,-,-,-,-,1.000,0.57
1,1,"Adames, C",SF,,542436,SS,7,5,46.0,15,...,8,0,2,-,-,-,-,-,1.000,2.14
2,1,"Adams, A",DET,,542866,P,15,0,16.2,4,...,3,0,0,-,-,-,-,-,1.000,0.27
3,1,"Adams, C",NYY,,664856,P,13,0,25.1,2,...,2,0,0,-,-,-,-,-,1.000,0.15
4,1,"Adduci, J",CHC,,451192,RF,1,1,8.0,5,...,0,0,0,-,-,-,-,-,1.000,5.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1319,"Walker, T",ARI,,592836,P,1,1,1.0,0,...,0,0,0,-,-,-,-,-,-,0.00
1,1319,"Warren, A",SEA,,605521,P,6,0,5.1,0,...,0,0,0,-,-,-,-,-,-,0.00
2,1319,"Williams, A",WSH,,657117,P,2,0,0.1,0,...,0,0,0,-,-,-,-,-,-,0.00
3,1319,"Williams, D",MIL,,642207,P,13,0,13.2,0,...,0,0,0,-,-,-,-,-,-,0.00


In [2]:
df_total_hit_ = pd.read_excel("df_total_hit.xlsx")
df_total_field_ = pd.read_excel("df_total_field.xlsx")

In [3]:
# 데이터 백업

player_hit_data = df_total_hit_
player_field_data = df_total_field_
player_hit_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1287 entries, 0 to 1286
Data columns (total 35 columns):
Unnamed: 0    1287 non-null int64
RK            1287 non-null int64
Player        1287 non-null object
Team          1287 non-null object
del           0 non-null float64
del2          1287 non-null int64
Pos           1287 non-null object
G             1287 non-null int64
AB            1287 non-null int64
R             1287 non-null int64
H             1287 non-null int64
2B            1287 non-null int64
3B            1287 non-null int64
HR            1287 non-null int64
RBI           1287 non-null int64
BB            1287 non-null int64
SO            1287 non-null int64
SB            1287 non-null int64
CS            1287 non-null int64
AVG           1287 non-null object
OBP           1287 non-null object
SLG           1287 non-null object
OPS           1287 non-null float64
IBB           1287 non-null int64
HBP           1287 non-null int64
SAC           1287 non-null int64
SF

In [4]:
# 불필요한 컬럼 삭제

player_hit_data = player_hit_data.drop(['Unnamed: 0', 'RK', 'del', 'del2', 'GO_AO', 'G'], axis=1)
player_field_data = player_field_data.drop(['Unnamed: 0', 'RK', 'del', 'del2', 'G', 'GS', 'SB', 'CS', 'SBPCT','PB','C_WP'], axis=1)

In [5]:
# column Player & Team $ Position 기준 병합

df_total = pd.merge(player_hit_data, player_field_data, on=['Player', 'Team', 'Pos']) # how='outer')
df_total.columns
#df_total.to_excel("test.xlsx")

Index(['Player', 'Team', 'Pos', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB',
       'SO', 'SB', 'CS', 'AVG', 'OBP', 'SLG', 'OPS', 'IBB', 'HBP', 'SAC', 'SF',
       'TB', 'XBH', 'GDP', 'GO', 'AO', 'NP', 'PA', 'INN', 'TC', 'PO', 'A', 'E',
       'DP', 'FPCT', 'RF'],
      dtype='object')

In [6]:
# 컬럼 조건문으로 행 추출하기 Position 컬럼에서 "1B", "2B", "3B", "SS"

# 하나의 조건문이라면 "해당 컬럼 == 하나조건"으로 하면 되지만, "해당컬럼.isin([여러조건])"
# df_total_hit_infld = df_total_hit[df_total_hit['Pos'] == "2B"]
df_total_infld = df_total[df_total['Pos'].isin(["1B", "2B", "3B", "SS"])]
df_total_infld
print(df_total_infld.shape)

(288, 37)


In [7]:
# df_total_infld.info()
# df_total_infld.dtypes 데이터 타입만 보려면

### 이렇게 해서 데이터 타입을 바꾸니 문제가 있네
```
# object --> numeric(int or float) 
# 현재 데이터는 전부 object type이여서 수치형 데이터는 pd.to_numeric함수를 이용해 변환
# 데이터프레임에서 컬럼으로 슬라이싱하려면 iloc을 사용
# df_total_infld['G_x'] = pd.to_numeric(df_total_infld['G_x'])
df_total_infld = df_total_infld.apply(pd.to_numeric.iloc, errors='coerce')

# 문자열 타입인데 numeric 타입으로 바뀐것을 원상 복구
# numeric(int or float) --> object
df_total_infld.astype({'Player':'str', 'Team':'str', 'Pos':'str'}).info()
```
- object가 numeric으로 바뀌니깐 NaN값이 되어 버림

- list로 해서 데이터 타입 일괄로 numeric으로 바꾸기
```
col_list = ['AB', 'R', 'H', '2B', '3B', 'HR', 'RBI',
       'BB', 'SO', 'SB', 'CS', 'AVG', 'OBP', 'SLG', 'OPS', 'IBB', 'HBP', 'SAC',
       'SF', 'TB', 'XBH', 'GDP', 'GO', 'AO', 'NP', 'PA', 'INN',
       'TC', 'PO', 'A', 'E', 'DP', 'FPCT', 'RF']
for i in col_list:
    df_total_infld[i] = pd.to_numeric(df_total_infld[i], errors='coerce')
```

In [8]:
# 타석수와 수비이닝의 최소값 제한

df_total_infld = df_total_infld[df_total_infld['AB'] >= 20]
print(df_total_infld.shape)
df_total_infld = df_total_infld[df_total_infld['INN'] >= 20]
print(df_total_infld.shape)

(274, 37)
(273, 37)


In [9]:
# Null 여부 확인
df_total_infld.isnull().count()

Player    273
Team      273
Pos       273
AB        273
R         273
H         273
2B        273
3B        273
HR        273
RBI       273
BB        273
SO        273
SB        273
CS        273
AVG       273
OBP       273
SLG       273
OPS       273
IBB       273
HBP       273
SAC       273
SF        273
TB        273
XBH       273
GDP       273
GO        273
AO        273
NP        273
PA        273
INN       273
TC        273
PO        273
A         273
E         273
DP        273
FPCT      273
RF        273
dtype: int64

- 특정 컬럼의 null값이 몇개 있는지 확인
```
df_total_infld.iloc[:,3:]
print(df_total_infld['FPCT'].isnull().value_counts())
```

- NaN값을 N으로 변경
```
df_total_infld['FPCT'].fillna('N', inplace=True)
df_total_infld.info()
```

## 데이터 분석하기
- groupby를 이용하여 분석하기


In [10]:
df_total_infld

,Player,Team,Pos,AB,R,H,2B,3B,HR,RBI,...,NP,PA,INN,TC,PO,A,E,DP,FPCT,RF
22,"Kendrick, H",WSH,3B,334,61,115,23,1,17,62,...,1397,370,601.0,401,308,91,2,35,.995,4.29
23,"Anderson, T",CWS,SS,498,81,167,32,0,18,56,...,1765,518,1050.0,532,182,324,26,75,.951,4.11
24,"Arraez, L",MIN,2B,326,54,109,20,1,4,28,...,1486,366,716.0,227,101,121,5,31,.978,2.22
37,"Solano, D",SF,SS,215,27,71,13,1,4,23,...,903,228,407.0,199,84,113,2,27,.990,3.46
40,"LeMahieu, D",NYY,2B,602,109,197,33,2,26,102,...,2463,655,1241.2,620,351,261,8,63,.987,3.64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
687,"Saladino, T",MIL,SS,65,7,8,0,0,2,8,...,287,71,151.0,62,28,33,1,7,.984,2.10
688,"Barreto, F",OAK,2B,57,6,7,2,0,2,5,...,237,58,146.2,71,20,46,5,8,.930,3.00
711,"Pedroia, D",BOS,2B,20,1,2,0,0,0,1,...,77,21,28.0,16,9,7,0,3,1.000,2.67
727,"Velazquez, A",CLE,3B,23,3,2,2,0,0,0,...,111,24,55.0,28,13,14,1,3,.964,2.25


In [11]:
# Playe와 Team 컬럼 삭제

df_total_infld.drop(['Player', 'Team'], axis=1, inplace=True)
df_total_infld.drop(['AB', 'INN'], axis=1, inplace=True)
df_total_infld.drop(['AVG', 'OBP', 'SLG', 'FPCT'], axis=1, inplace=True)

In [12]:
# 포지션 별로 각 수치 평균

df_total_infld.groupby('Pos').agg(['mean'])

,R,H,2B,3B,HR,RBI,BB,SO,SB,CS,...,GO,AO,NP,PA,TC,PO,A,E,DP,RF
,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,...,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean
Pos,,,,,,,,,,,,,,,,,,,,,
1B,43.692308,75.846154,15.892308,0.800000,15.000000,47.707692,37.061538,81.338462,1.323077,0.584615,...,72.000000,77.738462,1415.000000,347.969231,532.092308,479.738462,48.061538,4.292308,47.646154,5.406923
2B,37.118421,70.500000,13.434211,1.552632,8.947368,33.078947,23.144737,60.171053,4.500000,1.789474,...,72.578947,72.052632,1164.934211,301.776316,271.144737,117.710526,147.868421,5.565789,35.802632,2.968684
3B,42.810811,77.324324,16.256757,1.378378,13.013514,42.959459,29.445946,71.486486,2.716216,1.094595,...,70.945946,78.337838,1307.081081,331.243243,225.743243,95.445946,123.500000,6.797297,18.081081,2.554054
SS,46.034483,85.879310,17.758621,1.982759,11.741379,41.103448,24.982759,71.362069,6.706897,2.396552,...,84.500000,82.672414,1348.086207,351.810345,311.517241,103.655172,199.810345,8.051724,42.155172,3.162414


In [13]:
df_total_infld.drop(['OPS', 'RF'], axis=1, inplace=True)
df_total_infld.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 273 entries, 22 to 751
Data columns (total 27 columns):
Pos    273 non-null object
R      273 non-null int64
H      273 non-null int64
2B     273 non-null int64
3B     273 non-null int64
HR     273 non-null int64
RBI    273 non-null int64
BB     273 non-null int64
SO     273 non-null int64
SB     273 non-null int64
CS     273 non-null int64
IBB    273 non-null int64
HBP    273 non-null int64
SAC    273 non-null int64
SF     273 non-null int64
TB     273 non-null int64
XBH    273 non-null int64
GDP    273 non-null int64
GO     273 non-null int64
AO     273 non-null int64
NP     273 non-null int64
PA     273 non-null int64
TC     273 non-null int64
PO     273 non-null int64
A      273 non-null int64
E      273 non-null int64
DP     273 non-null int64
dtypes: int64(26), object(1)
memory usage: 69.7+ KB


In [14]:
# label 데이터 레이블 인코딩
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(df_total_infld['Pos'])
df_total_infld['Pos'] = encoder.transform(df_total_infld['Pos'])
#labels = labels.reshape(-1, 1)
df_total_infld

,Pos,R,H,2B,3B,HR,RBI,BB,SO,SB,...,GDP,GO,AO,NP,PA,TC,PO,A,E,DP
22,2,61,115,23,1,17,62,27,49,2,...,11,93,82,1397,370,401,308,91,2,35
23,3,81,167,32,0,18,56,15,109,17,...,12,131,93,1765,518,532,182,324,26,75
24,1,54,109,20,1,4,28,36,29,2,...,2,86,105,1486,366,227,101,121,5,31
37,3,27,71,13,1,4,23,10,49,0,...,4,43,54,903,228,199,84,113,2,27
40,1,109,197,33,2,26,102,46,90,5,...,14,184,136,2463,655,620,351,261,8,63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
687,3,7,8,0,0,2,8,5,26,2,...,1,16,15,287,71,62,28,33,1,7
688,1,6,7,2,0,2,5,1,23,1,...,0,18,9,237,58,71,20,46,5,8
711,1,1,2,0,0,0,1,1,2,0,...,2,9,7,77,21,16,9,7,0,3
727,2,3,2,2,0,0,0,1,13,1,...,0,4,4,111,24,28,13,14,1,3


In [15]:
# 데이터 세트 분리 (피처 데이터 and 라벨 데이터)

y_infield_df = df_total_infld['Pos']
x_infield_df = df_total_infld.drop('Pos', axis=1)

In [16]:
# train_test_split()를 통해 train과 test 데이터 분리
# from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_infield_df, y_infield_df, test_size=0.2, random_state=11)

In [17]:
print(x_infield_df.shape)
print(y_infield_df.shape)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(273, 26)
(273,)
(218, 26)
(218,)
(55, 26)
(55,)


# 1) 테스트로 여러가지 모델로 학습하고 예측/평가해보자
- Random Forest
- Decision Tree
- Gradient Boosting

In [38]:
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score
# from sklearn.ensemble import GradientBoostingClassifier
deci_tree = DecisionTreeClassifier(random_state=11)
rand_fore = RandomForestClassifier(random_state=11)
grad_boos = GradientBoostingClassifier(random_state=11)
xgb_wrap = XGBClassifier(n_estimators=500, learning_rate=0.03, max_depth=3)

# Decision Tree
deci_tree.fit(x_train, y_train)
deci_tree_pred = deci_tree.predict(x_test)
print('결정트리 정확도: {0:.3f}'.format(accuracy_score(y_test, deci_tree_pred)))

# Random Forest
rand_fore.fit(x_train, y_train)
rand_fore_pred = rand_fore.predict(x_test)
print('랜덤포레스트의 정확도: {0:.3f}'.format(accuracy_score(y_test, rand_fore_pred)))

# GradientBoosting
grad_boos.fit(x_train, y_train)
grad_boos_pred = grad_boos.predict(x_test)
print('GBM의 정확도: {0:.3f}'.format(accuracy_score(y_test, grad_boos_pred)))

# XGBClassifier
xgb_wrap.fit(x_train, y_train)
xgb_wrap_preds = xgb_wrap.predict(x_test)
print('XGB의 정확도: {0:.3f}'.format(accuracy_score(y_test, xgb_wrap_preds)))

결정트리 정확도: 0.491
랜덤포레스트의 정확도: 0.545
GBM의 정확도: 0.600
XGB의 정확도: 0.655


### 교차검증 : k폴드개로 나눠서 돌아가면서 검증
- cross_val_score()사용

## 1) 결정트리

In [203]:
# from sklearn.model_selection import cross_val_score

scores = cross_val_score(deci_tree, x_infield_df, y_infield_df, cv=4)
for iter, accuracy in enumerate(scores):
    print('교차 검증 {0} 정확도: {1:.4f}'.format(iter, accuracy))

print('교차 검증 평균 정확도 : {0:.4f}'.format(np.mean(scores)))

교차 검증 0 정확도: 0.6377
교차 검증 1 정확도: 0.6618
교차 검증 2 정확도: 0.5294
교차 검증 3 정확도: 0.4118
교차 검증 평균 정확도 : 0.5602


### 하이퍼파라미터 변화를 통한 트리 깊이 조절
- GridSearchCV : 하이퍼파라미터 조절

In [204]:
# from sklearn.model_selection import GridSearchCV

params = {
    'max_depth' : [4, 6, 8, 10, 12],
    'min_samples_split': [2,4,6,8,10,12]
}

grid_cv = GridSearchCV(deci_tree, param_grid=params, scoring='accuracy', cv=3, verbose=1)
grid_cv.fit(x_train, y_train)
print('GridSearchCV 최고 평균 정확도 수치:{0:.4f}'.format(grid_cv.best_score_))
print('GridSearchCV 최적 하이퍼 파라미터:', grid_cv.best_params_)

Fitting 3 folds for each of 30 candidates, totalling 90 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


GridSearchCV 최고 평균 정확도 수치:0.5829
GridSearchCV 최적 하이퍼 파라미터: {'max_depth': 8, 'min_samples_split': 6}


[Parallel(n_jobs=1)]: Done  90 out of  90 | elapsed:    0.8s finished


In [205]:
best_deci_tree = grid_cv.best_estimator_
pred1 = best_deci_tree.predict(x_test)
accuracy = accuracy_score(y_test, pred1)
print('결정 트리 예측 정확도:{0:.4f}'.format(accuracy))

결정 트리 예측 정확도:0.5818


## 2) 랜덤포레스트

In [206]:
scores = cross_val_score(rand_fore, x_infield_df, y_infield_df, cv=3)
for iter, accuracy in enumerate(scores):
    print('교차 검증 {0} 정확도: {1:.4f}'.format(iter, accuracy))

print('교차 검증 평균 정확도 : {0:.4f}'.format(np.mean(scores)))

교차 검증 0 정확도: 0.6703
교차 검증 1 정확도: 0.6484
교차 검증 2 정확도: 0.4505
교차 검증 평균 정확도 : 0.5897


In [208]:
# from sklearn.model_selection import GridSearchCV

params = {
    'max_depth' : [10, 12, 14],
    'min_samples_split': [2,4,6,8,10,12]
    #'min_samples_leaf' : [2,4,6,8,10],
    #'n_estimators' : [10]
}

grid_cv = GridSearchCV(rand_fore, param_grid=params, scoring='accuracy', cv=3, verbose=1)
grid_cv.fit(x_train, y_train)
print('GridSearchCV 최고 평균 정확도 수치:{0:.4f}'.format(grid_cv.best_score_))
print('GridSearchCV 최적 하이퍼 파라미터:', grid_cv.best_params_)

Fitting 3 folds for each of 18 candidates, totalling 54 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  54 out of  54 | elapsed:   15.6s finished


GridSearchCV 최고 평균 정확도 수치:0.5965
GridSearchCV 최적 하이퍼 파라미터: {'max_depth': 12, 'min_samples_split': 4}


In [209]:
best_rand_fore = grid_cv.best_estimator_
pred1 = best_deci_tree.predict(x_test)
accuracy = accuracy_score(y_test, pred1)
print('결정 트리 예측 정확도:{0:.4f}'.format(accuracy))

결정 트리 예측 정확도:0.5818


In [210]:
scores = cross_val_score(best_rand_fore, x_infield_df, y_infield_df, cv=3)
for iter, accuracy in enumerate(scores):
    print('교차 검증 {0} 정확도: {1:.4f}'.format(iter, accuracy))

print('교차 검증 평균 정확도 : {0:.4f}'.format(np.mean(scores)))

교차 검증 0 정확도: 0.6264
교차 검증 1 정확도: 0.6813
교차 검증 2 정확도: 0.4725
교차 검증 평균 정확도 : 0.5934


## 3) GBM

In [44]:
scores = cross_val_score(grad_boos, x_infield_df, y_infield_df, cv=4)
for iter, accuracy in enumerate(scores):
    print('교차 검증 {0} 정확도: {1:.4f}'.format(iter, accuracy))

print('교차 검증 평균 정확도 : {0:.4f}'.format(np.mean(scores)))

교차 검증 0 정확도: 0.6522
교차 검증 1 정확도: 0.6471
교차 검증 2 정확도: 0.5294
교차 검증 3 정확도: 0.5147
교차 검증 평균 정확도 : 0.5858


In [46]:
# from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators':[5, 10, 20, 30, 40, 50, 60, 70],
    'learning_rate':[0.01, 0.05, 0.06, 0.08],
}

grid_cv_g = GridSearchCV(grad_boos, param_grid=params, scoring='accuracy', cv=3, verbose=1)
grid_cv_g.fit(x_train, y_train)
print('GridSearchCV 최고 평균 정확도 수치:{0:.4f}'.format(grid_cv_g.best_score_))
print('GridSearchCV 최적 하이퍼 파라미터:', grid_cv_g.best_params_)

Fitting 3 folds for each of 32 candidates, totalling 96 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  96 out of  96 | elapsed:   30.2s finished


GridSearchCV 최고 평균 정확도 수치:0.6104
GridSearchCV 최적 하이퍼 파라미터: {'learning_rate': 0.06, 'n_estimators': 50}


In [47]:
best_grad_boos = grid_cv_g.best_estimator_
print(grid_cv_g.best_estimator_)
pred_g = best_grad_boos.predict(x_test)
accuracy = accuracy_score(y_test, pred_g)
print('결정 트리 예측 정확도:{0:.4f}'.format(accuracy))

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.06, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=50,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=11, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)
결정 트리 예측 정확도:0.5273


In [48]:
scores = cross_val_score(best_grad_boos, x_infield_df, y_infield_df, cv=3)
for iter, accuracy in enumerate(scores):
    print('교차 검증 {0} 정확도: {1:.4f}'.format(iter, accuracy))

print('교차 검증 평균 정확도 : {0:.4f}'.format(np.mean(scores)))

교차 검증 0 정확도: 0.6703
교차 검증 1 정확도: 0.6374
교차 검증 2 정확도: 0.4066
교차 검증 평균 정확도 : 0.5714


d
d

In [43]:
# from sklearn.model_selection import cross_val_score

scores = cross_val_score(xgb_wrap, x_infield_df, y_infield_df, cv=5)
for iter, accuracy in enumerate(scores):
    print('교차 검증 {0} 정확도: {1:.4f}'.format(iter, accuracy))

print('교차 검증 평균 정확도 : {0:.4f}'.format(np.mean(scores)))

교차 검증 0 정확도: 0.6909
교차 검증 1 정확도: 0.7818
교차 검증 2 정확도: 0.6000
교차 검증 3 정확도: 0.5556
교차 검증 4 정확도: 0.3889
교차 검증 평균 정확도 : 0.6034


In [49]:
# from sklearn.model_selection import GridSearchCV

params = {
    'max_depth' : [10, 12, 14],
    'min_samples_split': [2,4,6,8,10,12]
    #'min_samples_leaf' : [2,4,6,8,10],
    #'n_estimators' : [10]
}

grid_cv_x = GridSearchCV(xgb_wrap, param_grid=params, scoring='accuracy', cv=3, verbose=1)
grid_cv_x.fit(x_train, y_train)
print('GridSearchCV 최고 평균 정확도 수치:{0:.4f}'.format(grid_cv_x.best_score_))
print('GridSearchCV 최적 하이퍼 파라미터:', grid_cv_x.best_params_)

Fitting 3 folds for each of 18 candidates, totalling 54 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  54 out of  54 | elapsed:  1.6min finished


GridSearchCV 최고 평균 정확도 수치:0.5962
GridSearchCV 최적 하이퍼 파라미터: {'max_depth': 12, 'min_samples_split': 2}


In [ ]:
xgb_wrap = GridSearchCV(grad_boos, param_grid=params, scoring='accuracy', cv=3, verbose=1)
grid_cv_g.fit(x_train, y_train)
print('GridSearchCV 최고 평균 정확도 수치:{0:.4f}'.format(grid_cv_g.best_score_))
print('GridSearchCV 최적 하이퍼 파라미터:', grid_cv_g.best_params_)

- Random Forest 모델로 GridSearchCV의 best 파라미터를 찾고 그 파라미터로 교차검증을 하니 테스트 정확도가 **54.5% > 59.34%** 증가

# 2) 테스트로 딥러닝으로 학습하고 예측/평가해보자
- DNN

In [96]:
x_train, x_test, y_train, y_test = train_test_split(x_infield_df, y_infield_df, test_size=0.2, random_state=11)

In [97]:
print(x_infield_df.shape)
print(y_infield_df.shape)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(273, 26)
(273,)
(218, 26)
(218,)
(55, 26)
(55,)


In [98]:
import keras
from keras import models
from keras import layers

### 학습할 때 data의 편차가 커서 정규화 해준다.

In [99]:
mean = x_train.mean(axis=0)
x_train -= mean
std = x_train.std(axis=0)
x_train /= std

mean = x_test.mean(axis=0)
x_test -= mean
std = x_test.std(axis=0)
x_test /= std

In [100]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [101]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(26,)))
    model.add(layers.Dense(4, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    
    return model

def build_model_deep():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', kernel_initializer='he_normal', input_shape=(26,)))
    model.add(layers.Dense(128, activation='relu', kernel_initializer='he_normal'))
    model.add(layers.Dense(128, activation='relu', kernel_initializer='he_normal'))
    model.add(layers.Dense(4, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()

    return model

def build_model_deep2():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', kernel_initializer='he_normal', input_shape=(26,)))
    model.add(layers.Dense(128, activation='relu', kernel_initializer='he_normal'))
    model.add(layers.Dense(256, activation='relu', kernel_initializer='he_normal'))
    model.add(layers.Dense(512, activation='relu', kernel_initializer='he_normal'))
    model.add(layers.Dense(4, activation='softmax'))
    model.compile(optimizer='Adagrad', loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    
    return model

def build_model_deep3():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', kernel_initializer='he_normal', input_shape=(26,)))
    model.add(layers.Dense(128, activation='relu', kernel_initializer='he_normal'))
    model.add(layers.Dense(256, activation='relu', kernel_initializer='he_normal'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(512, activation='relu', kernel_initializer='he_normal'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(4, activation='softmax'))
    model.compile(optimizer='Adagrad', loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    
    return model

def build_model_deep4():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', kernel_initializer='he_normal', input_shape=(26,)))
    model.add(layers.Dense(128, activation='relu', kernel_initializer='he_normal'))
    model.add(layers.Dense(256, activation='relu', kernel_initializer='he_normal'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(512, activation='relu', kernel_initializer='he_normal'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(4, activation='softmax'))
    model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    
    return model

In [121]:
optimizer = ["Adam", "RMSProp","SGD"]
acc = []
batch_size = 32
for i in optimizer:
    model = build_model()
    model.fit(x_train, y_train, batch_size=batch_size, epochs=1000, shuffle=True)
    score = model.evaluate(x_test, y_test, verbose=0)
    # print('1-hidden layer 예측 정확도:{0:.4f}'.format(score[1]))
    acc.append(score[1])

Model: "sequential_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_70 (Dense)             (None, 64)                1728      
_________________________________________________________________
dense_71 (Dense)             (None, 4)                 260       
Total params: 1,988
Trainable params: 1,988
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000
218/218 [==============================] - 0s 2ms/step - loss: 1.4906 - accuracy: 0.2569
Epoch 2/1000
218/218 [==============================] - 0s 83us/step - loss: 1.3526 - accuracy: 0.3028
Epoch 3/1000
218/218 [==============================] - 0s 78us/step - loss: 1.2658 - accuracy: 0.4037
Epoch 4/1000
218/218 [==============================] - 0s 78us/step - loss: 1.2110 - accuracy: 0.4312
Epoch 5/1000
218/218 [==============================] - 0s 73us/step - loss: 1.1702 - accuracy: 0.4817

218/218 [==============================] - 0s 73us/step - loss: 0.4301 - accuracy: 0.8761
Epoch 151/1000
218/218 [==============================] - 0s 80us/step - loss: 0.4291 - accuracy: 0.8670
Epoch 152/1000
218/218 [==============================] - 0s 102us/step - loss: 0.4293 - accuracy: 0.8624
Epoch 153/1000
218/218 [==============================] - 0s 78us/step - loss: 0.4277 - accuracy: 0.8486
Epoch 154/1000
218/218 [==============================] - 0s 83us/step - loss: 0.4249 - accuracy: 0.8716
Epoch 155/1000
218/218 [==============================] - 0s 74us/step - loss: 0.4249 - accuracy: 0.8716
Epoch 156/1000
218/218 [==============================] - 0s 83us/step - loss: 0.4209 - accuracy: 0.8807
Epoch 157/1000
218/218 [==============================] - 0s 73us/step - loss: 0.4201 - accuracy: 0.8761
Epoch 158/1000
218/218 [==============================] - 0s 87us/step - loss: 0.4196 - accuracy: 0.8624
Epoch 159/1000
218/218 [==============================] - 0s 78us/ste

218/218 [==============================] - 0s 74us/step - loss: 0.2922 - accuracy: 0.9083
Epoch 305/1000
218/218 [==============================] - 0s 73us/step - loss: 0.2926 - accuracy: 0.8991
Epoch 306/1000
218/218 [==============================] - 0s 78us/step - loss: 0.2898 - accuracy: 0.8991
Epoch 307/1000
218/218 [==============================] - 0s 82us/step - loss: 0.2900 - accuracy: 0.8899
Epoch 308/1000
218/218 [==============================] - 0s 75us/step - loss: 0.2907 - accuracy: 0.8945
Epoch 309/1000
218/218 [==============================] - 0s 78us/step - loss: 0.2909 - accuracy: 0.8853
Epoch 310/1000
218/218 [==============================] - 0s 83us/step - loss: 0.2908 - accuracy: 0.8945
Epoch 311/1000
218/218 [==============================] - 0s 78us/step - loss: 0.2895 - accuracy: 0.9037
Epoch 312/1000
218/218 [==============================] - 0s 78us/step - loss: 0.2943 - accuracy: 0.8899
Epoch 313/1000
218/218 [==============================] - 0s 87us/step

218/218 [==============================] - 0s 83us/step - loss: 0.2226 - accuracy: 0.9312
Epoch 460/1000
218/218 [==============================] - 0s 73us/step - loss: 0.2229 - accuracy: 0.9174
Epoch 461/1000
218/218 [==============================] - 0s 82us/step - loss: 0.2258 - accuracy: 0.9128
Epoch 462/1000
218/218 [==============================] - 0s 78us/step - loss: 0.2207 - accuracy: 0.9220
Epoch 463/1000
218/218 [==============================] - 0s 87us/step - loss: 0.2231 - accuracy: 0.9037
Epoch 464/1000
218/218 [==============================] - 0s 83us/step - loss: 0.2210 - accuracy: 0.9220
Epoch 465/1000
218/218 [==============================] - 0s 75us/step - loss: 0.2218 - accuracy: 0.9220
Epoch 466/1000
218/218 [==============================] - 0s 83us/step - loss: 0.2252 - accuracy: 0.9037
Epoch 467/1000
218/218 [==============================] - 0s 83us/step - loss: 0.2221 - accuracy: 0.9083
Epoch 468/1000
218/218 [==============================] - 0s 78us/step

218/218 [==============================] - 0s 78us/step - loss: 0.1731 - accuracy: 0.9450
Epoch 614/1000
218/218 [==============================] - 0s 69us/step - loss: 0.1727 - accuracy: 0.9495
Epoch 615/1000
218/218 [==============================] - 0s 80us/step - loss: 0.1747 - accuracy: 0.9404
Epoch 616/1000
218/218 [==============================] - 0s 83us/step - loss: 0.1768 - accuracy: 0.9450
Epoch 617/1000
218/218 [==============================] - 0s 74us/step - loss: 0.1726 - accuracy: 0.9404
Epoch 618/1000
218/218 [==============================] - 0s 73us/step - loss: 0.1755 - accuracy: 0.9450
Epoch 619/1000
218/218 [==============================] - 0s 83us/step - loss: 0.1701 - accuracy: 0.9450
Epoch 620/1000
218/218 [==============================] - 0s 78us/step - loss: 0.1713 - accuracy: 0.9450
Epoch 621/1000
218/218 [==============================] - 0s 78us/step - loss: 0.1712 - accuracy: 0.9450
Epoch 622/1000
218/218 [==============================] - 0s 73us/step

...

218/218 [==============================] - 0s 78us/step - loss: 0.2798 - accuracy: 0.8899
Epoch 311/1000
218/218 [==============================] - 0s 87us/step - loss: 0.2765 - accuracy: 0.8899
Epoch 312/1000
218/218 [==============================] - 0s 83us/step - loss: 0.2759 - accuracy: 0.9083
Epoch 313/1000
218/218 [==============================] - 0s 80us/step - loss: 0.2775 - accuracy: 0.8899
Epoch 314/1000
218/218 [==============================] - 0s 87us/step - loss: 0.2730 - accuracy: 0.8945
Epoch 315/1000
218/218 [==============================] - 0s 74us/step - loss: 0.2745 - accuracy: 0.9037
Epoch 316/1000
218/218 [==============================] - 0s 73us/step - loss: 0.2735 - accuracy: 0.8945
Epoch 317/1000
218/218 [==============================] - 0s 75us/step - loss: 0.2737 - accuracy: 0.8853
Epoch 318/1000
218/218 [==============================] - 0s 74us/step - loss: 0.2777 - accuracy: 0.8761
Epoch 319/1000
218/218 [==============================] - 0s 73us/step

218/218 [==============================] - 0s 68us/step - loss: 0.2032 - accuracy: 0.9220
Epoch 466/1000
218/218 [==============================] - 0s 78us/step - loss: 0.2042 - accuracy: 0.9358
Epoch 467/1000
218/218 [==============================] - 0s 78us/step - loss: 0.2014 - accuracy: 0.9358
Epoch 468/1000
218/218 [==============================] - 0s 73us/step - loss: 0.2038 - accuracy: 0.9312
Epoch 469/1000
218/218 [==============================] - 0s 74us/step - loss: 0.2018 - accuracy: 0.9220
Epoch 470/1000
218/218 [==============================] - 0s 83us/step - loss: 0.2001 - accuracy: 0.9358
Epoch 471/1000
218/218 [==============================] - 0s 77us/step - loss: 0.2034 - accuracy: 0.9220
Epoch 472/1000
218/218 [==============================] - 0s 79us/step - loss: 0.1979 - accuracy: 0.9404
Epoch 473/1000
218/218 [==============================] - 0s 78us/step - loss: 0.1975 - accuracy: 0.9450
Epoch 474/1000
218/218 [==============================] - 0s 78us/step

218/218 [==============================] - 0s 101us/step - loss: 0.1529 - accuracy: 0.9541
Epoch 620/1000
218/218 [==============================] - 0s 87us/step - loss: 0.1519 - accuracy: 0.9541
Epoch 621/1000
218/218 [==============================] - 0s 106us/step - loss: 0.1531 - accuracy: 0.9587
Epoch 622/1000
218/218 [==============================] - 0s 96us/step - loss: 0.1523 - accuracy: 0.9587
Epoch 623/1000
218/218 [==============================] - 0s 86us/step - loss: 0.1567 - accuracy: 0.9495
Epoch 624/1000
218/218 [==============================] - 0s 92us/step - loss: 0.1532 - accuracy: 0.9495
Epoch 625/1000
218/218 [==============================] - 0s 87us/step - loss: 0.1518 - accuracy: 0.9541
Epoch 626/1000
218/218 [==============================] - 0s 87us/step - loss: 0.1528 - accuracy: 0.9587
Epoch 627/1000
218/218 [==============================] - 0s 92us/step - loss: 0.1510 - accuracy: 0.9541
Epoch 628/1000
218/218 [==============================] - 0s 92us/st

218/218 [==============================] - 0s 79us/step - loss: 0.1228 - accuracy: 0.9633
Epoch 774/1000
218/218 [==============================] - 0s 78us/step - loss: 0.1219 - accuracy: 0.9679
Epoch 775/1000
218/218 [==============================] - 0s 78us/step - loss: 0.1200 - accuracy: 0.9679
Epoch 776/1000
218/218 [==============================] - ETA: 0s - loss: 0.0896 - accuracy: 1.00 - 0s 79us/step - loss: 0.1185 - accuracy: 0.9679
Epoch 777/1000
218/218 [==============================] - 0s 83us/step - loss: 0.1178 - accuracy: 0.9679
Epoch 778/1000
218/218 [==============================] - 0s 73us/step - loss: 0.1189 - accuracy: 0.9679
Epoch 779/1000
218/218 [==============================] - 0s 78us/step - loss: 0.1197 - accuracy: 0.9679
Epoch 780/1000
218/218 [==============================] - 0s 80us/step - loss: 0.1184 - accuracy: 0.9725
Epoch 781/1000
218/218 [==============================] - 0s 73us/step - loss: 0.1230 - accuracy: 0.9633
Epoch 782/1000
218/218 [====

218/218 [==============================] - 0s 84us/step - loss: 0.0952 - accuracy: 0.9771
Epoch 929/1000
218/218 [==============================] - 0s 80us/step - loss: 0.0930 - accuracy: 0.9725
Epoch 930/1000
218/218 [==============================] - 0s 73us/step - loss: 0.0963 - accuracy: 0.9771
Epoch 931/1000
218/218 [==============================] - 0s 92us/step - loss: 0.0950 - accuracy: 0.9817
Epoch 932/1000
218/218 [==============================] - 0s 78us/step - loss: 0.0918 - accuracy: 0.9817
Epoch 933/1000
218/218 [==============================] - 0s 81us/step - loss: 0.0963 - accuracy: 0.9771
Epoch 934/1000
218/218 [==============================] - 0s 73us/step - loss: 0.0932 - accuracy: 0.9771
Epoch 935/1000
218/218 [==============================] - 0s 78us/step - loss: 0.0930 - accuracy: 0.9862
Epoch 936/1000
218/218 [==============================] - 0s 69us/step - loss: 0.0914 - accuracy: 0.9771
Epoch 937/1000
218/218 [==============================] - 0s 79us/step

In [122]:
print('Adam정확도:{0:.4f}, PMSProp정확도:{1:.4f}, SGD정확도{2:.4f}'.format(acc[0], acc[1], acc[2]))

Adam정확도:0.6000, PMSProp정확도:0.5273, SGD정확도0.5818



## Single hidden layer

In [103]:
model = build_model()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_32 (Dense)             (None, 64)                1728      
_________________________________________________________________
dense_33 (Dense)             (None, 4)                 260       
Total params: 1,988
Trainable params: 1,988
Non-trainable params: 0
_________________________________________________________________


In [104]:
batch_size = 16

In [105]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=2000, shuffle=True)

Epoch 1/2000
218/218 [==============================] - 0s 1ms/step - loss: 1.5663 - accuracy: 0.2339
Epoch 2/2000
218/218 [==============================] - 0s 115us/step - loss: 1.4035 - accuracy: 0.2982
Epoch 3/2000
218/218 [==============================] - 0s 110us/step - loss: 1.2972 - accuracy: 0.3716
Epoch 4/2000
218/218 [==============================] - 0s 115us/step - loss: 1.2139 - accuracy: 0.4679
Epoch 5/2000
218/218 [==============================] - 0s 119us/step - loss: 1.1477 - accuracy: 0.5229
Epoch 6/2000
218/218 [==============================] - 0s 99us/step - loss: 1.0997 - accuracy: 0.5459
Epoch 7/2000
218/218 [==============================] - 0s 115us/step - loss: 1.0589 - accuracy: 0.5688
Epoch 8/2000
218/218 [==============================] - ETA: 0s - loss: 1.0512 - accuracy: 0.56 - 0s 115us/step - loss: 1.0260 - accuracy: 0.5734
Epoch 9/2000
218/218 [==============================] - ETA: 0s - loss: 0.9653 - accuracy: 0.62 - 0s 115us/step - loss: 0.9989 - 

Epoch 78/2000
218/218 [==============================] - 0s 116us/step - loss: 0.5082 - accuracy: 0.8119
Epoch 79/2000
218/218 [==============================] - 0s 106us/step - loss: 0.5020 - accuracy: 0.8211
Epoch 80/2000
218/218 [==============================] - 0s 110us/step - loss: 0.4975 - accuracy: 0.8257
Epoch 81/2000
218/218 [==============================] - 0s 123us/step - loss: 0.4947 - accuracy: 0.8119
Epoch 82/2000
218/218 [==============================] - 0s 103us/step - loss: 0.4905 - accuracy: 0.8257
Epoch 83/2000
218/218 [==============================] - 0s 120us/step - loss: 0.4935 - accuracy: 0.8165
Epoch 84/2000
218/218 [==============================] - 0s 110us/step - loss: 0.4822 - accuracy: 0.8394
Epoch 85/2000
218/218 [==============================] - 0s 115us/step - loss: 0.4884 - accuracy: 0.8073
Epoch 86/2000
218/218 [==============================] - 0s 105us/step - loss: 0.4818 - accuracy: 0.8349
Epoch 87/2000
218/218 [==============================] 

218/218 [==============================] - 0s 128us/step - loss: 0.2758 - accuracy: 0.8945
Epoch 233/2000
218/218 [==============================] - 0s 123us/step - loss: 0.2734 - accuracy: 0.8991
Epoch 234/2000
218/218 [==============================] - 0s 114us/step - loss: 0.2685 - accuracy: 0.9174
Epoch 235/2000
218/218 [==============================] - 0s 105us/step - loss: 0.2706 - accuracy: 0.9083
Epoch 236/2000
218/218 [==============================] - 0s 106us/step - loss: 0.2722 - accuracy: 0.9083
Epoch 237/2000
218/218 [==============================] - 0s 96us/step - loss: 0.2710 - accuracy: 0.9083
Epoch 238/2000
218/218 [==============================] - 0s 93us/step - loss: 0.2826 - accuracy: 0.9037
Epoch 239/2000
218/218 [==============================] - 0s 93us/step - loss: 0.2674 - accuracy: 0.9037
Epoch 240/2000
218/218 [==============================] - 0s 92us/step - loss: 0.2694 - accuracy: 0.8991
Epoch 241/2000
218/218 [==============================] - 0s 97us

218/218 [==============================] - 0s 122us/step - loss: 0.1814 - accuracy: 0.9358
Epoch 387/2000
218/218 [==============================] - 0s 128us/step - loss: 0.1805 - accuracy: 0.9404
Epoch 388/2000
218/218 [==============================] - 0s 124us/step - loss: 0.1796 - accuracy: 0.9312
Epoch 389/2000
218/218 [==============================] - 0s 289us/step - loss: 0.1808 - accuracy: 0.9404
Epoch 390/2000
218/218 [==============================] - 0s 128us/step - loss: 0.1788 - accuracy: 0.9450
Epoch 391/2000
218/218 [==============================] - 0s 138us/step - loss: 0.1778 - accuracy: 0.9404
Epoch 392/2000
218/218 [==============================] - 0s 133us/step - loss: 0.1830 - accuracy: 0.9404
Epoch 393/2000
218/218 [==============================] - 0s 133us/step - loss: 0.1765 - accuracy: 0.9266
Epoch 394/2000
218/218 [==============================] - 0s 124us/step - loss: 0.1773 - accuracy: 0.9358
Epoch 395/2000
218/218 [==============================] - 0s 

218/218 [==============================] - 0s 108us/step - loss: 0.1249 - accuracy: 0.9771
Epoch 541/2000
218/218 [==============================] - 0s 105us/step - loss: 0.1294 - accuracy: 0.9450
Epoch 542/2000
218/218 [==============================] - 0s 115us/step - loss: 0.1286 - accuracy: 0.9679
Epoch 543/2000
218/218 [==============================] - 0s 125us/step - loss: 0.1219 - accuracy: 0.9725
Epoch 544/2000
218/218 [==============================] - 0s 96us/step - loss: 0.1244 - accuracy: 0.9679
Epoch 545/2000
218/218 [==============================] - 0s 97us/step - loss: 0.1194 - accuracy: 0.9771
Epoch 546/2000
218/218 [==============================] - 0s 106us/step - loss: 0.1230 - accuracy: 0.9679
Epoch 547/2000
218/218 [==============================] - 0s 97us/step - loss: 0.1207 - accuracy: 0.9679
Epoch 548/2000
218/218 [==============================] - 0s 115us/step - loss: 0.1345 - accuracy: 0.9587
Epoch 549/2000
218/218 [==============================] - 0s 107

...

Epoch 1383/2000
218/218 [==============================] - 0s 105us/step - loss: 0.0135 - accuracy: 1.0000
Epoch 1384/2000
218/218 [==============================] - 0s 100us/step - loss: 0.0142 - accuracy: 1.0000
Epoch 1385/2000
218/218 [==============================] - 0s 101us/step - loss: 0.0129 - accuracy: 1.0000
Epoch 1386/2000
218/218 [==============================] - 0s 101us/step - loss: 0.0143 - accuracy: 1.0000
Epoch 1387/2000
218/218 [==============================] - 0s 98us/step - loss: 0.0170 - accuracy: 0.9954
Epoch 1388/2000
218/218 [==============================] - 0s 102us/step - loss: 0.0177 - accuracy: 1.0000
Epoch 1389/2000
218/218 [==============================] - 0s 100us/step - loss: 0.0222 - accuracy: 0.9908
Epoch 1390/2000
218/218 [==============================] - 0s 106us/step - loss: 0.0209 - accuracy: 0.9954
Epoch 1391/2000
218/218 [==============================] - 0s 101us/step - loss: 0.0154 - accuracy: 1.0000
Epoch 1392/2000
218/218 [=============

218/218 [==============================] - 0s 133us/step - loss: 0.0093 - accuracy: 1.0000
Epoch 1536/2000
218/218 [==============================] - 0s 128us/step - loss: 0.0093 - accuracy: 1.0000
Epoch 1537/2000
218/218 [==============================] - 0s 128us/step - loss: 0.0096 - accuracy: 1.0000
Epoch 1538/2000
218/218 [==============================] - 0s 119us/step - loss: 0.0102 - accuracy: 1.0000
Epoch 1539/2000
218/218 [==============================] - 0s 124us/step - loss: 0.0107 - accuracy: 1.0000
Epoch 1540/2000
218/218 [==============================] - 0s 124us/step - loss: 0.0099 - accuracy: 1.0000
Epoch 1541/2000
218/218 [==============================] - 0s 110us/step - loss: 0.0109 - accuracy: 1.0000
Epoch 1542/2000
218/218 [==============================] - 0s 124us/step - loss: 0.0116 - accuracy: 1.0000
Epoch 1543/2000
218/218 [==============================] - 0s 138us/step - loss: 0.0124 - accuracy: 1.0000
Epoch 1544/2000
218/218 [============================

Epoch 1687/2000
218/218 [==============================] - 0s 110us/step - loss: 0.0070 - accuracy: 1.0000
Epoch 1688/2000
218/218 [==============================] - 0s 128us/step - loss: 0.0076 - accuracy: 1.0000
Epoch 1689/2000
218/218 [==============================] - 0s 128us/step - loss: 0.0066 - accuracy: 1.0000
Epoch 1690/2000
218/218 [==============================] - 0s 133us/step - loss: 0.0068 - accuracy: 1.0000
Epoch 1691/2000
218/218 [==============================] - 0s 133us/step - loss: 0.0071 - accuracy: 1.0000
Epoch 1692/2000
218/218 [==============================] - 0s 119us/step - loss: 0.0069 - accuracy: 1.0000
Epoch 1693/2000
218/218 [==============================] - 0s 119us/step - loss: 0.0067 - accuracy: 1.0000
Epoch 1694/2000
218/218 [==============================] - 0s 110us/step - loss: 0.0062 - accuracy: 1.0000
Epoch 1695/2000
218/218 [==============================] - 0s 174us/step - loss: 0.0060 - accuracy: 1.0000
Epoch 1696/2000
218/218 [============

218/218 [==============================] - 0s 101us/step - loss: 0.0049 - accuracy: 1.0000
Epoch 1840/2000
218/218 [==============================] - 0s 119us/step - loss: 0.0047 - accuracy: 1.0000
Epoch 1841/2000
218/218 [==============================] - 0s 119us/step - loss: 0.0057 - accuracy: 1.0000
Epoch 1842/2000
218/218 [==============================] - 0s 117us/step - loss: 0.0050 - accuracy: 1.0000
Epoch 1843/2000
218/218 [==============================] - 0s 105us/step - loss: 0.0050 - accuracy: 1.0000
Epoch 1844/2000
218/218 [==============================] - 0s 124us/step - loss: 0.0047 - accuracy: 1.0000
Epoch 1845/2000
218/218 [==============================] - 0s 115us/step - loss: 0.0047 - accuracy: 1.0000
Epoch 1846/2000
218/218 [==============================] - 0s 106us/step - loss: 0.0048 - accuracy: 1.0000
Epoch 1847/2000
218/218 [==============================] - 0s 110us/step - loss: 0.0046 - accuracy: 1.0000
Epoch 1848/2000
218/218 [============================

218/218 [==============================] - 0s 143us/step - loss: 0.0037 - accuracy: 1.0000
Epoch 1992/2000
218/218 [==============================] - 0s 142us/step - loss: 0.0040 - accuracy: 1.0000
Epoch 1993/2000
218/218 [==============================] - 0s 147us/step - loss: 0.0039 - accuracy: 1.0000
Epoch 1994/2000
218/218 [==============================] - 0s 165us/step - loss: 0.0034 - accuracy: 1.0000
Epoch 1995/2000
218/218 [==============================] - 0s 133us/step - loss: 0.0041 - accuracy: 1.0000
Epoch 1996/2000
218/218 [==============================] - 0s 128us/step - loss: 0.0040 - accuracy: 1.0000
Epoch 1997/2000
218/218 [==============================] - 0s 133us/step - loss: 0.0037 - accuracy: 1.0000
Epoch 1998/2000
218/218 [==============================] - 0s 133us/step - loss: 0.0053 - accuracy: 1.0000
Epoch 1999/2000
218/218 [==============================] - 0s 138us/step - loss: 0.0054 - accuracy: 1.0000
Epoch 2000/2000
218/218 [============================

In [106]:
pred_keras = model.predict(x_test, verbose=0)

In [107]:
score = model.evaluate(x_test, y_test, verbose=0)
print('1-hidden layer 예측 정확도:{0:.4f}'.format(score[1]))

1-hidden layer 예측 정확도:0.5273


## Multi-hidden layer

In [62]:
model1 = build_model_deep()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 64)                1728      
_________________________________________________________________
dense_12 (Dense)             (None, 128)               8320      
_________________________________________________________________
dense_13 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_14 (Dense)             (None, 4)                 516       
Total params: 27,076
Trainable params: 27,076
Non-trainable params: 0
_________________________________________________________________


In [63]:
batch_size = 16
model1.fit(x_train, y_train, batch_size=batch_size, epochs=5000, shuffle=True)

Epoch 1/5000
218/218 [==============================] - 0s 2ms/step - loss: 1.5252 - accuracy: 0.3578
Epoch 2/5000
218/218 [==============================] - 0s 165us/step - loss: 1.0900 - accuracy: 0.5000
Epoch 3/5000
218/218 [==============================] - 0s 156us/step - loss: 0.9295 - accuracy: 0.5963
Epoch 4/5000
218/218 [==============================] - 0s 142us/step - loss: 0.8919 - accuracy: 0.6468
Epoch 5/5000
218/218 [==============================] - 0s 156us/step - loss: 0.7793 - accuracy: 0.6972
Epoch 6/5000
218/218 [==============================] - 0s 142us/step - loss: 0.7355 - accuracy: 0.7477
Epoch 7/5000
218/218 [==============================] - 0s 138us/step - loss: 0.6849 - accuracy: 0.7339
Epoch 8/5000
218/218 [==============================] - 0s 158us/step - loss: 0.6485 - accuracy: 0.7752
Epoch 9/5000
218/218 [==============================] - 0s 174us/step - loss: 0.6274 - accuracy: 0.7477
Epoch 10/5000
218/218 [==============================] - 0s 147us/

218/218 [==============================] - 0s 142us/step - loss: 0.1610 - accuracy: 0.9587
Epoch 156/5000
218/218 [==============================] - 0s 128us/step - loss: 0.0692 - accuracy: 0.9817
Epoch 157/5000
218/218 [==============================] - 0s 133us/step - loss: 0.0364 - accuracy: 0.9954
Epoch 158/5000
218/218 [==============================] - 0s 119us/step - loss: 0.0342 - accuracy: 0.9908
Epoch 159/5000
218/218 [==============================] - 0s 128us/step - loss: 0.0371 - accuracy: 0.9954
Epoch 160/5000
218/218 [==============================] - 0s 124us/step - loss: 0.0228 - accuracy: 1.0000
Epoch 161/5000
218/218 [==============================] - 0s 115us/step - loss: 0.0297 - accuracy: 0.9954
Epoch 162/5000
218/218 [==============================] - 0s 112us/step - loss: 0.0235 - accuracy: 0.9954
Epoch 163/5000
218/218 [==============================] - 0s 124us/step - loss: 0.0219 - accuracy: 1.0000
Epoch 164/5000
218/218 [==============================] - 0s 

218/218 [==============================] - 0s 151us/step - loss: 0.0013 - accuracy: 1.0000
Epoch 310/5000
218/218 [==============================] - 0s 119us/step - loss: 0.0014 - accuracy: 1.0000
Epoch 311/5000
218/218 [==============================] - 0s 123us/step - loss: 0.0014 - accuracy: 1.0000
Epoch 312/5000
218/218 [==============================] - 0s 133us/step - loss: 0.0014 - accuracy: 1.0000
Epoch 313/5000
218/218 [==============================] - 0s 133us/step - loss: 0.0013 - accuracy: 1.0000
Epoch 314/5000
218/218 [==============================] - 0s 148us/step - loss: 0.0013 - accuracy: 1.0000
Epoch 315/5000
218/218 [==============================] - 0s 128us/step - loss: 0.0012 - accuracy: 1.0000
Epoch 316/5000
218/218 [==============================] - 0s 118us/step - loss: 0.0012 - accuracy: 1.0000
Epoch 317/5000
218/218 [==============================] - 0s 115us/step - loss: 0.0012 - accuracy: 1.0000
Epoch 318/5000
218/218 [==============================] - 0s 

218/218 [==============================] - 0s 156us/step - loss: 3.1328e-04 - accuracy: 1.0000
Epoch 459/5000
218/218 [==============================] - 0s 143us/step - loss: 2.8974e-04 - accuracy: 1.0000
Epoch 460/5000
218/218 [==============================] - 0s 156us/step - loss: 3.0242e-04 - accuracy: 1.0000
Epoch 461/5000
218/218 [==============================] - 0s 151us/step - loss: 3.1914e-04 - accuracy: 1.0000
Epoch 462/5000
218/218 [==============================] - 0s 170us/step - loss: 3.1955e-04 - accuracy: 1.0000
Epoch 463/5000
218/218 [==============================] - 0s 161us/step - loss: 3.0073e-04 - accuracy: 1.0000
Epoch 464/5000
218/218 [==============================] - 0s 138us/step - loss: 3.6725e-04 - accuracy: 1.0000
Epoch 465/5000
218/218 [==============================] - 0s 124us/step - loss: 3.5068e-04 - accuracy: 1.0000
Epoch 466/5000
218/218 [==============================] - 0s 133us/step - loss: 2.7683e-04 - accuracy: 1.0000
Epoch 467/5000
218/218 [=

218/218 [==============================] - 0s 161us/step - loss: 0.0016 - accuracy: 1.0000
Epoch 610/5000
218/218 [==============================] - 0s 161us/step - loss: 0.0016 - accuracy: 1.0000
Epoch 611/5000
218/218 [==============================] - 0s 155us/step - loss: 0.0016 - accuracy: 1.0000
Epoch 612/5000
218/218 [==============================] - 0s 151us/step - loss: 0.0015 - accuracy: 1.0000
Epoch 613/5000
218/218 [==============================] - 0s 115us/step - loss: 0.0015 - accuracy: 1.0000
Epoch 614/5000
218/218 [==============================] - 0s 138us/step - loss: 0.0015 - accuracy: 1.0000
Epoch 615/5000
218/218 [==============================] - 0s 123us/step - loss: 0.0014 - accuracy: 1.0000
Epoch 616/5000
218/218 [==============================] - 0s 121us/step - loss: 0.0014 - accuracy: 1.0000
Epoch 617/5000
218/218 [==============================] - 0s 119us/step - loss: 0.0014 - accuracy: 1.0000
Epoch 618/5000
218/218 [==============================] - 0s 

...

Epoch 4429/5000
218/218 [==============================] - 0s 126us/step - loss: 1.1921e-07 - accuracy: 1.0000
Epoch 4430/5000
218/218 [==============================] - 0s 126us/step - loss: 1.1921e-07 - accuracy: 1.0000
Epoch 4431/5000
218/218 [==============================] - 0s 131us/step - loss: 1.1921e-07 - accuracy: 1.0000
Epoch 4432/5000
218/218 [==============================] - 0s 130us/step - loss: 1.1921e-07 - accuracy: 1.0000
Epoch 4433/5000
218/218 [==============================] - 0s 124us/step - loss: 1.1921e-07 - accuracy: 1.0000
Epoch 4434/5000
218/218 [==============================] - 0s 124us/step - loss: 1.1921e-07 - accuracy: 1.0000
Epoch 4435/5000
218/218 [==============================] - 0s 120us/step - loss: 1.1921e-07 - accuracy: 1.0000
Epoch 4436/5000
218/218 [==============================] - 0s 114us/step - loss: 1.1921e-07 - accuracy: 1.0000
Epoch 4437/5000
218/218 [==============================] - 0s 115us/step - loss: 1.1921e-07 - accuracy: 1.0000
E

218/218 [==============================] - 0s 128us/step - loss: 1.1921e-07 - accuracy: 1.0000
Epoch 4576/5000
218/218 [==============================] - 0s 133us/step - loss: 1.1921e-07 - accuracy: 1.0000
Epoch 4577/5000
218/218 [==============================] - 0s 137us/step - loss: 1.1921e-07 - accuracy: 1.0000
Epoch 4578/5000
218/218 [==============================] - 0s 131us/step - loss: 1.1921e-07 - accuracy: 1.0000
Epoch 4579/5000
218/218 [==============================] - 0s 156us/step - loss: 1.1921e-07 - accuracy: 1.0000
Epoch 4580/5000
218/218 [==============================] - 0s 139us/step - loss: 1.1921e-07 - accuracy: 1.0000
Epoch 4581/5000
218/218 [==============================] - 0s 133us/step - loss: 1.1921e-07 - accuracy: 1.0000
Epoch 4582/5000
218/218 [==============================] - 0s 129us/step - loss: 1.1921e-07 - accuracy: 1.0000
Epoch 4583/5000
218/218 [==============================] - 0s 142us/step - loss: 1.1921e-07 - accuracy: 1.0000
Epoch 4584/5000
2

Epoch 4649/5000
218/218 [==============================] - 0s 119us/step - loss: 1.1921e-07 - accuracy: 1.0000
Epoch 4650/5000
218/218 [==============================] - 0s 115us/step - loss: 1.1921e-07 - accuracy: 1.0000
Epoch 4651/5000
218/218 [==============================] - 0s 126us/step - loss: 1.1921e-07 - accuracy: 1.0000
Epoch 4652/5000
218/218 [==============================] - ETA: 0s - loss: 1.1921e-07 - accuracy: 1.00 - 0s 133us/step - loss: 1.1921e-07 - accuracy: 1.0000
Epoch 4653/5000
218/218 [==============================] - 0s 118us/step - loss: 1.1921e-07 - accuracy: 1.0000
Epoch 4654/5000
218/218 [==============================] - 0s 119us/step - loss: 1.1921e-07 - accuracy: 1.0000
Epoch 4655/5000
218/218 [==============================] - 0s 129us/step - loss: 1.1921e-07 - accuracy: 1.0000
Epoch 4656/5000
218/218 [==============================] - 0s 122us/step - loss: 1.1921e-07 - accuracy: 1.0000
Epoch 4657/5000
218/218 [==============================] - 0s 127u

218/218 [==============================] - 0s 133us/step - loss: 1.1921e-07 - accuracy: 1.0000
Epoch 4796/5000
218/218 [==============================] - 0s 134us/step - loss: 1.1921e-07 - accuracy: 1.0000
Epoch 4797/5000
218/218 [==============================] - 0s 143us/step - loss: 1.1921e-07 - accuracy: 1.0000
Epoch 4798/5000
218/218 [==============================] - 0s 129us/step - loss: 1.1921e-07 - accuracy: 1.0000
Epoch 4799/5000
218/218 [==============================] - 0s 111us/step - loss: 1.1921e-07 - accuracy: 1.0000
Epoch 4800/5000
218/218 [==============================] - 0s 125us/step - loss: 1.1921e-07 - accuracy: 1.0000
Epoch 4801/5000
218/218 [==============================] - 0s 128us/step - loss: 1.1921e-07 - accuracy: 1.0000
Epoch 4802/5000
218/218 [==============================] - 0s 128us/step - loss: 1.1921e-07 - accuracy: 1.0000
Epoch 4803/5000
218/218 [==============================] - 0s 123us/step - loss: 1.1921e-07 - accuracy: 1.0000
Epoch 4804/5000
2

Epoch 4869/5000
218/218 [==============================] - 0s 142us/step - loss: 1.1921e-07 - accuracy: 1.0000
Epoch 4870/5000
218/218 [==============================] - 0s 140us/step - loss: 1.1921e-07 - accuracy: 1.0000
Epoch 4871/5000
218/218 [==============================] - 0s 138us/step - loss: 1.1921e-07 - accuracy: 1.0000
Epoch 4872/5000
218/218 [==============================] - 0s 152us/step - loss: 1.1921e-07 - accuracy: 1.0000
Epoch 4873/5000
218/218 [==============================] - 0s 143us/step - loss: 1.1921e-07 - accuracy: 1.0000
Epoch 4874/5000
218/218 [==============================] - 0s 142us/step - loss: 1.1921e-07 - accuracy: 1.0000
Epoch 4875/5000
218/218 [==============================] - 0s 155us/step - loss: 1.1921e-07 - accuracy: 1.0000
Epoch 4876/5000
218/218 [==============================] - 0s 133us/step - loss: 1.1921e-07 - accuracy: 1.0000
Epoch 4877/5000
218/218 [==============================] - 0s 124us/step - loss: 1.1921e-07 - accuracy: 1.0000
E

In [64]:
pred_keras1 = model1.predict(x_test, verbose=0)

In [65]:
score1 = model1.evaluate(x_test, y_test, verbose=0)
print('1-hidden layer 예측 정확도:{0:.4f}'.format(score1[1]))

1-hidden layer 예측 정확도:0.6364


## Multi-hidden layer2
- optimizer 변경
- hidden layer 추가

In [68]:
model1 = build_model_deep2()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 64)                1728      
_________________________________________________________________
dense_16 (Dense)             (None, 128)               8320      
_________________________________________________________________
dense_17 (Dense)             (None, 256)               33024     
_________________________________________________________________
dense_18 (Dense)             (None, 512)               131584    
_________________________________________________________________
dense_19 (Dense)             (None, 4)                 2052      
Total params: 176,708
Trainable params: 176,708
Non-trainable params: 0
_________________________________________________________________


In [69]:
batch_size = 5
model1.fit(x_train, y_train, batch_size=batch_size, epochs=3000, shuffle=True)

Epoch 1/3000
218/218 [==============================] - 0s 2ms/step - loss: 2.1418 - accuracy: 0.4312
Epoch 2/3000
218/218 [==============================] - 0s 828us/step - loss: 0.8330 - accuracy: 0.6743
Epoch 3/3000
218/218 [==============================] - 0s 720us/step - loss: 0.6486 - accuracy: 0.7248
Epoch 4/3000
218/218 [==============================] - 0s 642us/step - loss: 0.5079 - accuracy: 0.7936
Epoch 5/3000
218/218 [==============================] - 0s 642us/step - loss: 0.4487 - accuracy: 0.8165
Epoch 6/3000
218/218 [==============================] - 0s 647us/step - loss: 0.3871 - accuracy: 0.8716
Epoch 7/3000
218/218 [==============================] - 0s 774us/step - loss: 0.3686 - accuracy: 0.8532
Epoch 8/3000
218/218 [==============================] - 0s 647us/step - loss: 0.3431 - accuracy: 0.8624
Epoch 9/3000
218/218 [==============================] - 0s 632us/step - loss: 0.3036 - accuracy: 0.8670
Epoch 10/3000
218/218 [==============================] - 0s 683us/

218/218 [==============================] - 0s 647us/step - loss: 0.0126 - accuracy: 1.0000
Epoch 80/3000
218/218 [==============================] - 0s 574us/step - loss: 0.0111 - accuracy: 1.0000
Epoch 81/3000
218/218 [==============================] - 0s 578us/step - loss: 0.0114 - accuracy: 1.0000
Epoch 82/3000
218/218 [==============================] - 0s 624us/step - loss: 0.0093 - accuracy: 1.0000
Epoch 83/3000
218/218 [==============================] - 0s 614us/step - loss: 0.0103 - accuracy: 1.0000
Epoch 84/3000
218/218 [==============================] - 0s 652us/step - loss: 0.0116 - accuracy: 1.0000
Epoch 85/3000
218/218 [==============================] - 0s 582us/step - loss: 0.0093 - accuracy: 1.0000
Epoch 86/3000
218/218 [==============================] - 0s 675us/step - loss: 0.0126 - accuracy: 1.0000
Epoch 87/3000
218/218 [==============================] - 0s 587us/step - loss: 0.0092 - accuracy: 1.0000
Epoch 88/3000
218/218 [==============================] - 0s 615us/ste

218/218 [==============================] - 0s 768us/step - loss: 9.5080e-04 - accuracy: 1.0000
Epoch 233/3000
218/218 [==============================] - 0s 634us/step - loss: 9.0524e-04 - accuracy: 1.0000
Epoch 234/3000
218/218 [==============================] - 0s 675us/step - loss: 9.5704e-04 - accuracy: 1.0000
Epoch 235/3000
218/218 [==============================] - 0s 698us/step - loss: 9.2853e-04 - accuracy: 1.0000
Epoch 236/3000
218/218 [==============================] - 0s 771us/step - loss: 9.0576e-04 - accuracy: 1.0000
Epoch 237/3000
218/218 [==============================] - 0s 656us/step - loss: 9.2789e-04 - accuracy: 1.0000
Epoch 238/3000
218/218 [==============================] - 0s 760us/step - loss: 9.0319e-04 - accuracy: 1.0000
Epoch 239/3000
218/218 [==============================] - 0s 688us/step - loss: 8.9100e-04 - accuracy: 1.0000
Epoch 240/3000
218/218 [==============================] - 0s 671us/step - loss: 8.8287e-04 - accuracy: 1.0000
Epoch 241/3000
218/218 [=

218/218 [==============================] - 0s 588us/step - loss: 4.1552e-04 - accuracy: 1.0000
Epoch 381/3000
218/218 [==============================] - 0s 633us/step - loss: 4.2034e-04 - accuracy: 1.0000
Epoch 382/3000
218/218 [==============================] - 0s 633us/step - loss: 4.1085e-04 - accuracy: 1.0000
Epoch 383/3000
218/218 [==============================] - 0s 537us/step - loss: 4.0942e-04 - accuracy: 1.0000
Epoch 384/3000
218/218 [==============================] - 0s 615us/step - loss: 4.0542e-04 - accuracy: 1.0000
Epoch 385/3000
218/218 [==============================] - 0s 619us/step - loss: 4.1170e-04 - accuracy: 1.0000
Epoch 386/3000
218/218 [==============================] - 0s 665us/step - loss: 4.0668e-04 - accuracy: 1.0000
Epoch 387/3000
218/218 [==============================] - 0s 656us/step - loss: 4.0281e-04 - accuracy: 1.0000
Epoch 388/3000
218/218 [==============================] - 0s 605us/step - loss: 4.0638e-04 - accuracy: 1.0000
Epoch 389/3000
218/218 [=

218/218 [==============================] - 0s 565us/step - loss: 2.4849e-04 - accuracy: 1.0000
Epoch 529/3000
218/218 [==============================] - 0s 583us/step - loss: 2.4820e-04 - accuracy: 1.0000
Epoch 530/3000
218/218 [==============================] - 0s 605us/step - loss: 2.4729e-04 - accuracy: 1.0000
Epoch 531/3000
218/218 [==============================] - 0s 583us/step - loss: 2.4595e-04 - accuracy: 1.0000
Epoch 532/3000
218/218 [==============================] - 0s 596us/step - loss: 2.4721e-04 - accuracy: 1.0000
Epoch 533/3000
218/218 [==============================] - 0s 568us/step - loss: 2.4511e-04 - accuracy: 1.0000
Epoch 534/3000
218/218 [==============================] - 0s 584us/step - loss: 2.4540e-04 - accuracy: 1.0000
Epoch 535/3000
218/218 [==============================] - 0s 570us/step - loss: 2.4343e-04 - accuracy: 1.0000
Epoch 536/3000
218/218 [==============================] - 0s 578us/step - loss: 2.4270e-04 - accuracy: 1.0000
Epoch 537/3000
218/218 [=

...

Epoch 2287/3000
218/218 [==============================] - 0s 547us/step - loss: 3.1334e-05 - accuracy: 1.0000
Epoch 2288/3000
218/218 [==============================] - 0s 583us/step - loss: 3.1319e-05 - accuracy: 1.0000
Epoch 2289/3000
218/218 [==============================] - 0s 564us/step - loss: 3.1433e-05 - accuracy: 1.0000
Epoch 2290/3000
218/218 [==============================] - 0s 588us/step - loss: 3.1308e-05 - accuracy: 1.0000
Epoch 2291/3000
218/218 [==============================] - 0s 617us/step - loss: 3.1306e-05 - accuracy: 1.0000
Epoch 2292/3000
218/218 [==============================] - 0s 604us/step - loss: 3.1260e-05 - accuracy: 1.0000
Epoch 2293/3000
218/218 [==============================] - 0s 615us/step - loss: 3.1249e-05 - accuracy: 1.0000
Epoch 2294/3000
218/218 [==============================] - 0s 609us/step - loss: 3.1232e-05 - accuracy: 1.0000
Epoch 2295/3000
218/218 [==============================] - 0s 568us/step - loss: 3.1222e-05 - accuracy: 1.0000
E

218/218 [==============================] - 0s 572us/step - loss: 2.8865e-05 - accuracy: 1.0000
Epoch 2434/3000
218/218 [==============================] - 0s 578us/step - loss: 2.8880e-05 - accuracy: 1.0000
Epoch 2435/3000
218/218 [==============================] - 0s 576us/step - loss: 2.8874e-05 - accuracy: 1.0000
Epoch 2436/3000
218/218 [==============================] - 0s 541us/step - loss: 2.8861e-05 - accuracy: 1.0000
Epoch 2437/3000
218/218 [==============================] - 0s 573us/step - loss: 2.8751e-05 - accuracy: 1.0000
Epoch 2438/3000
218/218 [==============================] - 0s 573us/step - loss: 2.8838e-05 - accuracy: 1.0000
Epoch 2439/3000
218/218 [==============================] - 0s 564us/step - loss: 2.8789e-05 - accuracy: 1.0000
Epoch 2440/3000
218/218 [==============================] - 0s 568us/step - loss: 2.8766e-05 - accuracy: 1.0000
Epoch 2441/3000
218/218 [==============================] - 0s 620us/step - loss: 2.8810e-05 - accuracy: 1.0000
Epoch 2442/3000
2

218/218 [==============================] - 0s 579us/step - loss: 2.6751e-05 - accuracy: 1.0000
Epoch 2580/3000
218/218 [==============================] - 0s 587us/step - loss: 2.6714e-05 - accuracy: 1.0000
Epoch 2581/3000
218/218 [==============================] - 0s 587us/step - loss: 2.6731e-05 - accuracy: 1.0000
Epoch 2582/3000
218/218 [==============================] - 0s 560us/step - loss: 2.6737e-05 - accuracy: 1.0000
Epoch 2583/3000
218/218 [==============================] - 0s 573us/step - loss: 2.6709e-05 - accuracy: 1.0000
Epoch 2584/3000
218/218 [==============================] - 0s 596us/step - loss: 2.6691e-05 - accuracy: 1.0000
Epoch 2585/3000
218/218 [==============================] - 0s 596us/step - loss: 2.6662e-05 - accuracy: 1.0000
Epoch 2586/3000
218/218 [==============================] - 0s 592us/step - loss: 2.6631e-05 - accuracy: 1.0000
Epoch 2587/3000
218/218 [==============================] - 0s 660us/step - loss: 2.6629e-05 - accuracy: 1.0000
Epoch 2588/3000
2

218/218 [==============================] - 0s 583us/step - loss: 2.4884e-05 - accuracy: 1.0000
Epoch 2726/3000
218/218 [==============================] - 0s 555us/step - loss: 2.4866e-05 - accuracy: 1.0000
Epoch 2727/3000
218/218 [==============================] - 0s 578us/step - loss: 2.4855e-05 - accuracy: 1.00000s - loss: 2.5420e-05 - accuracy: 1.
Epoch 2728/3000
218/218 [==============================] - 0s 596us/step - loss: 2.4820e-05 - accuracy: 1.0000
Epoch 2729/3000
218/218 [==============================] - 0s 596us/step - loss: 2.4839e-05 - accuracy: 1.0000
Epoch 2730/3000
218/218 [==============================] - 0s 587us/step - loss: 2.4797e-05 - accuracy: 1.0000
Epoch 2731/3000
218/218 [==============================] - 0s 564us/step - loss: 2.4837e-05 - accuracy: 1.0000
Epoch 2732/3000
218/218 [==============================] - 0s 601us/step - loss: 2.4807e-05 - accuracy: 1.0000
Epoch 2733/3000
218/218 [==============================] - 0s 543us/step - loss: 2.4744e-05 

Epoch 2872/3000
218/218 [==============================] - 0s 637us/step - loss: 2.3156e-05 - accuracy: 1.0000
Epoch 2873/3000
218/218 [==============================] - 0s 674us/step - loss: 2.3103e-05 - accuracy: 1.0000
Epoch 2874/3000
218/218 [==============================] - 0s 578us/step - loss: 2.3096e-05 - accuracy: 1.0000
Epoch 2875/3000
218/218 [==============================] - 0s 610us/step - loss: 2.3064e-05 - accuracy: 1.0000
Epoch 2876/3000
218/218 [==============================] - 0s 620us/step - loss: 2.3083e-05 - accuracy: 1.0000
Epoch 2877/3000
218/218 [==============================] - 0s 588us/step - loss: 2.3068e-05 - accuracy: 1.0000
Epoch 2878/3000
218/218 [==============================] - 0s 573us/step - loss: 2.3040e-05 - accuracy: 1.0000
Epoch 2879/3000
218/218 [==============================] - 0s 605us/step - loss: 2.3044e-05 - accuracy: 1.0000
Epoch 2880/3000
218/218 [==============================] - 0s 610us/step - loss: 2.2993e-05 - accuracy: 1.0000
E

In [70]:
score1 = model1.evaluate(x_test, y_test, verbose=0)
print('1-hidden layer 예측 정확도:{0:.4f}'.format(score1[1]))

1-hidden layer 예측 정확도:0.5455


### Multi-hidden layer3
- dropout 추가

In [76]:
model3 = build_model_deep3()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_23 (Dense)             (None, 64)                1728      
_________________________________________________________________
dense_24 (Dense)             (None, 128)               8320      
_________________________________________________________________
dense_25 (Dense)             (None, 256)               33024     
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_26 (Dense)             (None, 512)               131584    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_27 (Dense)             (None, 4)                

In [77]:
batch_size = 5
model3.fit(x_train, y_train, batch_size=batch_size, epochs=3000, shuffle=True)

Epoch 1/3000
218/218 [==============================] - 1s 3ms/step - loss: 3.0158 - accuracy: 0.3486
Epoch 2/3000
218/218 [==============================] - 0s 890us/step - loss: 1.2742 - accuracy: 0.48170s - loss: 1.3559 - accuracy: 0.
Epoch 3/3000
218/218 [==============================] - 0s 942us/step - loss: 1.0622 - accuracy: 0.5092
Epoch 4/3000
218/218 [==============================] - 0s 891us/step - loss: 1.0163 - accuracy: 0.5596
Epoch 5/3000
218/218 [==============================] - 0s 1ms/step - loss: 0.8879 - accuracy: 0.5917
Epoch 6/3000
218/218 [==============================] - 0s 1ms/step - loss: 0.9277 - accuracy: 0.6009
Epoch 7/3000
218/218 [==============================] - 0s 1ms/step - loss: 0.8754 - accuracy: 0.6193
Epoch 8/3000
218/218 [==============================] - 0s 913us/step - loss: 0.7883 - accuracy: 0.6514
Epoch 9/3000
218/218 [==============================] - 0s 835us/step - loss: 0.8463 - accuracy: 0.6697
Epoch 10/3000
218/218 [=================

218/218 [==============================] - 0s 700us/step - loss: 0.0724 - accuracy: 0.9817
Epoch 155/3000
218/218 [==============================] - 0s 720us/step - loss: 0.0712 - accuracy: 0.9771
Epoch 156/3000
218/218 [==============================] - 0s 651us/step - loss: 0.0624 - accuracy: 0.9908
Epoch 157/3000
218/218 [==============================] - 0s 739us/step - loss: 0.0444 - accuracy: 0.9954
Epoch 158/3000
218/218 [==============================] - 0s 684us/step - loss: 0.0442 - accuracy: 0.9908
Epoch 159/3000
218/218 [==============================] - 0s 679us/step - loss: 0.0521 - accuracy: 0.9908
Epoch 160/3000
218/218 [==============================] - 0s 674us/step - loss: 0.0696 - accuracy: 0.9771
Epoch 161/3000
218/218 [==============================] - 0s 656us/step - loss: 0.0563 - accuracy: 0.9817
Epoch 162/3000
218/218 [==============================] - 0s 784us/step - loss: 0.0968 - accuracy: 0.9633
Epoch 163/3000
218/218 [==============================] - 0s 

218/218 [==============================] - 0s 840us/step - loss: 0.0485 - accuracy: 0.9862
Epoch 232/3000
218/218 [==============================] - 0s 853us/step - loss: 0.0295 - accuracy: 0.9908
Epoch 233/3000
218/218 [==============================] - 0s 797us/step - loss: 0.0318 - accuracy: 0.9908
Epoch 234/3000
218/218 [==============================] - 0s 879us/step - loss: 0.0300 - accuracy: 0.9954
Epoch 235/3000
218/218 [==============================] - 0s 1ms/step - loss: 0.0208 - accuracy: 0.9954
Epoch 236/3000
218/218 [==============================] - 0s 811us/step - loss: 0.0300 - accuracy: 1.0000
Epoch 237/3000
218/218 [==============================] - 0s 830us/step - loss: 0.0358 - accuracy: 0.99080s - loss: 0.0410 - accuracy: 0.
Epoch 238/3000
218/218 [==============================] - 0s 757us/step - loss: 0.0354 - accuracy: 0.9862
Epoch 239/3000
218/218 [==============================] - 0s 853us/step - loss: 0.0279 - accuracy: 0.9954
Epoch 240/3000
218/218 [=======

218/218 [==============================] - 0s 761us/step - loss: 0.0119 - accuracy: 1.0000
Epoch 383/3000
218/218 [==============================] - 0s 875us/step - loss: 0.0124 - accuracy: 1.0000
Epoch 384/3000
218/218 [==============================] - 0s 889us/step - loss: 0.0120 - accuracy: 1.0000
Epoch 385/3000
218/218 [==============================] - 0s 995us/step - loss: 0.0144 - accuracy: 0.9954
Epoch 386/3000
218/218 [==============================] - 0s 1ms/step - loss: 0.0138 - accuracy: 1.0000
Epoch 387/3000
218/218 [==============================] - 0s 1ms/step - loss: 0.0111 - accuracy: 1.0000
Epoch 388/3000
218/218 [==============================] - 0s 985us/step - loss: 0.0129 - accuracy: 0.9954
Epoch 389/3000
218/218 [==============================] - 0s 1ms/step - loss: 0.0095 - accuracy: 1.0000
Epoch 390/3000
218/218 [==============================] - 0s 1ms/step - loss: 0.0157 - accuracy: 0.9954
Epoch 391/3000
218/218 [==============================] - 0s 927us/st

218/218 [==============================] - 0s 616us/step - loss: 0.0079 - accuracy: 1.0000
Epoch 537/3000
218/218 [==============================] - 0s 633us/step - loss: 0.0025 - accuracy: 1.0000
Epoch 538/3000
218/218 [==============================] - 0s 780us/step - loss: 0.0046 - accuracy: 1.0000
Epoch 539/3000
218/218 [==============================] - 0s 638us/step - loss: 0.0036 - accuracy: 1.0000
Epoch 540/3000
218/218 [==============================] - 0s 656us/step - loss: 0.0036 - accuracy: 1.0000
Epoch 541/3000
218/218 [==============================] - 0s 647us/step - loss: 0.0060 - accuracy: 1.0000
Epoch 542/3000
218/218 [==============================] - 0s 633us/step - loss: 0.0085 - accuracy: 1.0000
Epoch 543/3000
218/218 [==============================] - 0s 767us/step - loss: 0.0120 - accuracy: 1.0000
Epoch 544/3000
218/218 [==============================] - 0s 665us/step - loss: 0.0048 - accuracy: 1.0000
Epoch 545/3000
218/218 [==============================] - 0s 

...

218/218 [==============================] - 0s 789us/step - loss: 6.9391e-04 - accuracy: 1.0000
Epoch 2347/3000
218/218 [==============================] - 0s 775us/step - loss: 6.2978e-04 - accuracy: 1.0000
Epoch 2348/3000
218/218 [==============================] - 0s 826us/step - loss: 3.9832e-04 - accuracy: 1.0000
Epoch 2349/3000
218/218 [==============================] - 0s 855us/step - loss: 0.0010 - accuracy: 1.0000
Epoch 2350/3000
218/218 [==============================] - 0s 1ms/step - loss: 7.7694e-04 - accuracy: 1.0000
Epoch 2351/3000
218/218 [==============================] - 0s 899us/step - loss: 3.3141e-04 - accuracy: 1.0000
Epoch 2352/3000
218/218 [==============================] - 0s 862us/step - loss: 8.2711e-04 - accuracy: 1.0000
Epoch 2353/3000
218/218 [==============================] - 0s 768us/step - loss: 6.0972e-04 - accuracy: 1.0000
Epoch 2354/3000
218/218 [==============================] - 0s 876us/step - loss: 5.2254e-04 - accuracy: 1.0000
Epoch 2355/3000
218/218

218/218 [==============================] - 0s 687us/step - loss: 3.5864e-04 - accuracy: 1.0000
Epoch 2494/3000
218/218 [==============================] - 0s 771us/step - loss: 0.0022 - accuracy: 1.0000
Epoch 2495/3000
218/218 [==============================] - 0s 638us/step - loss: 4.7258e-04 - accuracy: 1.0000
Epoch 2496/3000
218/218 [==============================] - 0s 661us/step - loss: 2.4715e-04 - accuracy: 1.0000
Epoch 2497/3000
218/218 [==============================] - 0s 702us/step - loss: 4.8858e-04 - accuracy: 1.0000
Epoch 2498/3000
218/218 [==============================] - 0s 661us/step - loss: 7.2588e-04 - accuracy: 1.0000
Epoch 2499/3000
218/218 [==============================] - 0s 656us/step - loss: 0.0028 - accuracy: 1.0000
Epoch 2500/3000
218/218 [==============================] - 0s 684us/step - loss: 3.7735e-04 - accuracy: 1.0000
Epoch 2501/3000
218/218 [==============================] - 0s 711us/step - loss: 0.0018 - accuracy: 1.0000
Epoch 2502/3000
218/218 [====

218/218 [==============================] - 0s 900us/step - loss: 3.4157e-04 - accuracy: 1.0000
Epoch 2642/3000
218/218 [==============================] - 0s 814us/step - loss: 0.0018 - accuracy: 1.0000
Epoch 2643/3000
218/218 [==============================] - 0s 660us/step - loss: 2.3787e-04 - accuracy: 1.0000
Epoch 2644/3000
218/218 [==============================] - 0s 789us/step - loss: 0.0026 - accuracy: 1.0000
Epoch 2645/3000
218/218 [==============================] - 0s 757us/step - loss: 4.7578e-04 - accuracy: 1.0000
Epoch 2646/3000
218/218 [==============================] - 0s 817us/step - loss: 0.0018 - accuracy: 1.0000
Epoch 2647/3000
218/218 [==============================] - 0s 835us/step - loss: 1.9679e-04 - accuracy: 1.0000
Epoch 2648/3000
218/218 [==============================] - 0s 755us/step - loss: 3.7140e-04 - accuracy: 1.0000
Epoch 2649/3000
218/218 [==============================] - 0s 716us/step - loss: 0.0084 - accuracy: 0.9954
Epoch 2650/3000
218/218 [========

218/218 [==============================] - 0s 642us/step - loss: 3.8913e-04 - accuracy: 1.0000
Epoch 2789/3000
218/218 [==============================] - 0s 697us/step - loss: 5.9013e-04 - accuracy: 1.0000
Epoch 2790/3000
218/218 [==============================] - 0s 683us/step - loss: 3.5546e-04 - accuracy: 1.0000
Epoch 2791/3000
218/218 [==============================] - 0s 706us/step - loss: 2.5994e-04 - accuracy: 1.0000
Epoch 2792/3000
218/218 [==============================] - 0s 684us/step - loss: 0.0011 - accuracy: 1.0000
Epoch 2793/3000
218/218 [==============================] - 0s 692us/step - loss: 1.8084e-04 - accuracy: 1.0000
Epoch 2794/3000
218/218 [==============================] - 0s 675us/step - loss: 2.4145e-04 - accuracy: 1.0000
Epoch 2795/3000
218/218 [==============================] - 0s 656us/step - loss: 2.0531e-04 - accuracy: 1.0000
Epoch 2796/3000
218/218 [==============================] - 0s 688us/step - loss: 4.7062e-04 - accuracy: 1.0000
Epoch 2797/3000
218/2

218/218 [==============================] - 0s 685us/step - loss: 1.7071e-04 - accuracy: 1.0000
Epoch 2937/3000
218/218 [==============================] - 0s 730us/step - loss: 5.9670e-04 - accuracy: 1.0000
Epoch 2938/3000
218/218 [==============================] - 0s 688us/step - loss: 5.4409e-04 - accuracy: 1.0000
Epoch 2939/3000
218/218 [==============================] - 0s 656us/step - loss: 1.5041e-04 - accuracy: 1.0000
Epoch 2940/3000
218/218 [==============================] - 0s 819us/step - loss: 9.7404e-04 - accuracy: 1.0000
Epoch 2941/3000
218/218 [==============================] - 0s 682us/step - loss: 4.6682e-04 - accuracy: 1.0000
Epoch 2942/3000
218/218 [==============================] - 0s 697us/step - loss: 7.4631e-04 - accuracy: 1.0000
Epoch 2943/3000
218/218 [==============================] - 0s 660us/step - loss: 0.0017 - accuracy: 1.0000
Epoch 2944/3000
218/218 [==============================] - 0s 1ms/step - loss: 5.2708e-04 - accuracy: 1.0000
Epoch 2945/3000
218/218

In [78]:
score3 = model3.evaluate(x_test, y_test, verbose=0)
print('1-hidden layer 예측 정확도:{0:.4f}'.format(score3[1]))

1-hidden layer 예측 정확도:0.5818
